# Entendendo LLMs

Este notebook apresenta uma introdução às LLMs (Large Language Model) utilizando a biblioteca transformers.

## Sumário

1. Checar existência de GPU
2. Carregar LLM da biblioteca trasnformers
3. Checar informações da LLM
4. Criar função que processa um prompt com a LLM


---


# 1. Checar existência de GPU

Ao carregar um LLM da biblioteca transformers, precisamos informar se a máquina possui CPU ou GPU.

Utilizando o comando "nvidia-smi" é possível checar a existência de GPU na máquina - caso não haja, o comando não funcionará.  Caso haja GPU, é possível checar outras informações sobre a performance da máquina.

Observação: O comando "nvidia-smi" é executado na command line do sistema, NÃO no python. No Google Colab, é necessário colocar um ponto de exclamação ("!") na frente de comandos a serem executados na cmd.

In [ ]:
!nvidia-smi

Fri Sep 13 02:18:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

---


# 2. Carregar LLM da biblioteca trasnformers

Carregamos nosso modelo utilizando o "pipeline" da biblioteca transformers. O "pipeline" serve para agrupar todas as funções necessárias ao processar a entrada e gerar a saída do prompt. Utilizaremos apenas 3 parâmetros: task (a função que o modelo executa); model (nome do modelo carregado); device (tipo de máquina utilizada, podendo ser CPU='cpu' ou GPU='cuda').

O modelo escolhido para essa introdução foi o TinyLlama ('TinyLlama/TinyLlama-1.1B-Chat-v1.0'), um modelo textual generativo básico.

In [ ]:
# Fazendo os imports necessários:
import transformers

# Carregando a LLM TinyLlama através do pipeline:
modelo = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device='cuda' # 'cuda' = GPU
)

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

---


# 3. Checar informações da LLM

Após carregar o modelo, podemos trazer mais detalhes sobre seu funcionamento. Para isso, utilizamos o método "model" da variável em que carregamos o modelo.

In [ ]:
modelo.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

Observando as informações trazidas, concluímos que o modelo:

*   Tem 22 blocos de processamento;
*   Faz normalização;
*   Suporta até 2048 tokens de input;
*   Funciona transformando o texto do input em tokens com o tokenizer e retornando a sequência de palavras mais provável.  


---


# 4. Criar função que processa um prompt com a LLM

Com o modelo carregado, podemos, finalmente, testá-lo. Criamos uma função que recebe o modelo carregado e o prompt que será processado, e printa na tela o prompt, sua codificação em tokens, a quantidade de tokens, a decodificação dos tokens, os primeiros tokens e o output gerado pelo modelo.

Ao processar o prompt com o modelo, existe um parâmetro chamado "temperature". Ele age como o nível de "criativadade" do modelo. Na prática, o que ele faz é aumentar a variedade da sequência de palavras geradas. O valor inserido no parâmetro irá alterar as probabilidades de sequência de cada palavra, possibilitando palavras, originalmente menos prováveis, de serem utilizadas.

Observação: Para utilizar o parâmetro "temperature", também é necessário setar o parâmetro "do_sample"=True.

In [ ]:
def processa_prompt(modelo, prompt):
  print("="*80)

  # Codificando o prompt em tokens
  tokens = modelo.tokenizer.encode(prompt, return_tensors='pt')
  print(f"- prompt original: {prompt}")
  print(f"- tokens: {tokens}")
  print(f"- # de tokens: {len(tokens)}")

  # Decodificando os tokens
  decoded = modelo.tokenizer.convert_ids_to_tokens(tokens[0])
  print(f"- tokens decodificados: {decoded}")

  first_tokens = modelo.tokenizer.convert_ids_to_tokens(range(5))
  print(f"- primeiros tokens: {first_tokens}")

  # Processando o prompt com o modelo
  # O nível de "criatividade" do modelo é alterado para 0.7 através do parâmetro "temperature"
  # É necessário setar "do_sample"=True para controlar a temperatura
  output = modelo(prompt, do_sample=True, temperature=0.7)
  print(f"- output do modelo: {output}")
  print(f"- texto gerado efetivamente: {output[0]['generated_text']}")

  print("="*80)
  return

processa_prompt(modelo, "Once upon a time")

- prompt original: Once upon a time
- tokens: tensor([[   1, 9038, 2501,  263,  931]])
- # de tokens: 1
- tokens decodificados: ['<s>', '▁Once', '▁upon', '▁a', '▁time']
- primeiros tokens: ['<unk>', '<s>', '</s>', '<0x00>', '<0x01>']
- output do modelo: [{'generated_text': 'Once upon a time, there was a young woman named Lily. Lily was a shy and introverted person who lived in a small town. One day, Lily met a man named Jack. Jack was an adventurous and outgoing person who lived in the city. Lily and Jack fell in love, but they knew it was never going to work out between them. One day, while walking in the forest, Jack and Lily stumbled upon a hidden cave. Inside the cave, they found a beautiful crystal, which sparked their curiosity. Lily and Jack started talking about the crystal, and Jack revealed that he had discovered the cave years ago and that the crystal was a rare gem. Jack showed Lily some of the other stones in the cave and told her that they could make a fortune if only the